Il file ingr_map.pkl prevede la struttura con le seguenti colonne:
-   raw_ingr: forma originale
-   raw_words: numero di parole nella forma originale
-   processed: forma processata
-   len_proc: lunghezza caratteri della forma processata
-   replaced: ingrediente con cui si fa la sostituzione 
-   count: conteggio della frequenza del dato ingrediente
-   id: identificativo unico dell'ingrediente

Il file è stato pulito mantenendo le colonne:
-   raw_ingr: forma originale
-   replaced: ingrediente con cui si fa la sostituzione 
-   id: identificativo unico dell'ingrediente
e salvato in Ingredients.json

Dal file calories.csv (fornito dal dataset Ingredients and calories) abbiamo generato distinct_category.txt, in modo da avere le categorie presenti nel file. Per ogni categoria, col supporto di ChatGPT, sono stati generati due valori, quantity_min e quantity_max, in modo da poter determinare un range verosimile di quantità per ogni ingrediente da usare in una ricetta. 


Partendo dal file calories.csv (fornito dal dataset Ingredients and calories) abbiamo pulito la struttura generando il file cleaned_calories.csv, eliminando duplicati (sulla base dell'attributo FoodItem), il quale presenta le seguenti colonne: 
-   FoodCategory
-   FoodItem
-   per100grams
-   Cals_per100grams
-   KJ_per100grams
-   quantity_min
-   quantity_max

A questo punto, utilizzando il seguente script, abbiamo effettuato il matching per verosimiglianza, tra i file Ingredients.json e cleaned_calories.csv effettuando il matching tra le replaced (Ingredients.json) e FoodItem (cleaned_calories.csv).

In [ ]:
from fuzzywuzzy import process
import json
import pandas as pd

# Load the JSON file
with open('Ingredients.json', 'r') as file:
    ingredients_data = json.load(file)

# Load the CSV file
updated_calories_df = pd.read_csv('cleaned_calories.csv')

# Displaying the first few records of each to understand their structure
ingredients_sample = ingredients_data[:5]
updated_calories_sample = updated_calories_df.head()

ingredients_sample, updated_calories_sample

# Creating a DataFrame from the JSON data
ingredients_df = pd.DataFrame(ingredients_data)

# Renaming the columns for clarity and ease of merging
updated_calories_df.rename(columns={'FoodItem': 'replaced'}, inplace=True)

# Creating a list of unique 'replaced' values from the ingredients DataFrame
unique_replaced_ingredients = ingredients_df['replaced'].unique()

print("Number of unique 'replaced' values in the ingredients DataFrame: ", len(unique_replaced_ingredients))

# Creating a dictionary to map each 'replaced' value to the most similar 'replaced' value in the updated_calories DataFrame
replaced_mapping = {replaced: process.extractOne(replaced, updated_calories_df['replaced'].unique())[0] 
                    for replaced in unique_replaced_ingredients}

# Mapping the 'replaced' values in ingredients DataFrame
ingredients_df['foodItem'] = ingredients_df['replaced'].map(replaced_mapping)

# Merging the DataFrames on the new 'foodItem' column in ingredients_df and 'replaced' in updated_calories_df
merged_fuzzy_df = pd.merge(ingredients_df, updated_calories_df, left_on='foodItem', right_on='replaced', how='left')

# Keeping both 'replaced' columns for clarity
merged_fuzzy_df.rename(columns={'replaced_x': 'replaced_ingredients', 'replaced_y': 'replaced_calories'}, inplace=True)

# Counting the number of records in the new merged DataFrame
new_record_count = merged_fuzzy_df.shape[0]

print("Number of records in the new merged DataFrame: ", new_record_count)

print(merged_fuzzy_df.head())

# save the new DataFrame to a CSV file
merged_fuzzy_df.to_csv('merged_fuzzy_df.csv', index=False)

In [ ]:
df = pd.read_csv('merged_fuzzy_df.csv')

# print to json file
df.to_json('merged_fuzzy_df.json', orient='records')

Il codice nel file ingr_merger.py esegue le seguenti operazioni:

Importazione di moduli: Importa i moduli fuzzywuzzy, json e pandas. fuzzywuzzy è utilizzato per il matching di stringhe basato sulla somiglianza, mentre pandas è per la manipolazione dei dati e json per la lettura di file JSON.

Caricamento del file JSON: Carica un file JSON chiamato Ingredients.json, che probabilmente contiene dati sugli ingredienti.

Caricamento del file CSV: Carica un file CSV chiamato cleaned_calories.csv, presumibilmente contenente dati sulle calorie.

Visualizzazione di campioni di dati: Visualizza i primi cinque record di ciascun set di dati per comprendere la loro struttura.

Creazione di DataFrame da dati JSON: Converte i dati JSON in un DataFrame di pandas.

Rinominazione delle colonne: Rinomina le colonne nel DataFrame CSV per chiarezza e per facilitare l'unione dei dati.

Estrazione di valori unici: Crea una lista di valori unici della colonna replaced del DataFrame degli ingredienti.

Stampa del numero di valori unici: Stampa il numero di valori unici nella colonna replaced del DataFrame degli ingredienti.

Mappatura dei valori 'replaced': Utilizza fuzzywuzzy per mappare ciascun valore di replaced nel DataFrame degli ingredienti al valore più simile nel DataFrame delle calorie.

Mappatura e unione dei DataFrame: Mappa i valori di replaced nel DataFrame degli ingredienti e poi unisce i due DataFrame basandosi su questa mappatura.

Rinominazione delle colonne dopo l'unione: Rinomina le colonne nel DataFrame unito per chiarezza.

Conta dei record nel nuovo DataFrame unito: Conta il numero di record nel DataFrame risultante dall'unione.

Stampa dei primi record del nuovo DataFrame unito: Visualizza i primi record del DataFrame unito.

Salvataggio del nuovo DataFrame in un file CSV: Salva il DataFrame unito in un file CSV chiamato merged_fuzzy_df.csv.

La struttura dell'output finale, basata su questo codice, sarà un DataFrame pandas che unisce i dati degli ingredienti con i dati delle calorie, utilizzando la somiglianza delle stringhe per associare gli elementi corrispondenti. Questo DataFrame viene poi salvato in un file CSV. Questo approccio è utile per abbinare dati provenienti da fonti diverse che potrebbero avere modi leggermente diversi di riferirsi agli stessi elementi, come varianti di nomi di ingredienti.

Ho creato uno script che rimuove le colonne replaced_calories, per100grams e KJ_per100grams dal DataFrame ottenuto dal file merged_fuzzy_df.json e salva i dati rimanenti in un nuovo file JSON con formattazione pretty nel file merged_fuzzy_df_reduced.json

In [ ]:
# Eliminate le colonne specificate dal DataFrame
columns_to_drop = ['replaced_calories', 'per100grams', 'KJ_per100grams']
reduced_df = merged_fuzzy_df.drop(columns=columns_to_drop)

# Definizione del percorso del nuovo file JSON
new_json_path = 'merged_fuzzy_df_reduced.json'

# Salvataggio del DataFrame in un nuovo file JSON con formattazione pretty
reduced_df.to_json(new_json_path, orient='records', lines=False, indent=4)

Ho modificato la colonna Cals_per100grams per convertire i valori da stringhe nel formato "value cal" a numeri. Il DataFrame aggiornato è stato salvato in un nuovo file JSON nel file merged_fuzzy_df_updated_reduced.json

In [ ]:
# Conversione della colonna 'Cals_per100grams' da stringa a numero
reduced_df['Cals_per100grams'] = reduced_df['Cals_per100grams'].str.extract(r'(\d+)').astype(float)

# Salvataggio del DataFrame modificato in un nuovo file JSON
updated_json_path = 'updated_merged_fuzzy_df.json'
reduced_df.to_json(updated_json_path, orient='records', lines=False, indent=4)